<a href="https://colab.research.google.com/github/praju4105/prajwal/blob/main/Threat%20Intel%20Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install feedparser transformers torch

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.8 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sh

This part of the code is for Extracting, Pre Processing and Parsing of the feed and extracting CVE ids.

In [2]:
import feedparser
import re
import sqlite3
import torch
from transformers import pipeline, AutoTokenizer
from datetime import datetime

# Load AI Model & Tokenizer
MODEL_NAME = "distilbert-base-uncased"
classifier = pipeline("text-classification", model=MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Threat Intelligence Feeds
FEEDS = [
  "https://feeds.feedburner.com/TheHackersNews"
]

# Connect to SQLite Database
conn = sqlite3.connect("threat_intel.db")
cursor = conn.cursor()

# Create Table with Non-CVE Tagging
cursor.execute("""
    CREATE TABLE IF NOT EXISTS threats (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        link TEXT,
        published TEXT,
        description TEXT,
        cve_id TEXT,
        risk_label TEXT,
        risk_score REAL,
        non_cve_tag TEXT,
        category TEXT
    )
""")
conn.commit()
print("✅ Database setup completed.")

# Precompile CVE Regex Patterns
CVE_PATTERN = re.compile(r"CVE-\d{4}-\d{4,7}")
URL_CVE_PATTERN = re.compile(r"cve\.org.*?CVE-\d{4}-\d{4,7}")

def extract_cves(text):
    """Extracts CVEs from text and returns them as a list."""
    cve_ids = CVE_PATTERN.findall(text)
    url_cve_ids = URL_CVE_PATTERN.findall(text)
    return list(set(cve_ids + url_cve_ids))

def preprocess_text(text):
    """Tokenizes and summarizes text if necessary."""
    tokens = tokenizer.tokenize(text)
    if len(tokens) > 512:
        try:
            summary = summarizer(text, max_length=150, min_length=50, do_sample=False)
            return summary[0]['summary_text']
        except Exception as e:
            print("Summerized")
    return text

def classify_threat(text):
    """Classifies the threat risk using AI."""
    try:
        result = classifier(text)[0]
        return result["label"], round(result["score"], 4)
    except Exception as e:
        return "Unknown", 0.0

def classify_non_cve_threats(text):
    """Classifies non-CVE threats based on patterns."""
    patterns = {
        "SQL Injection": r"\b(sql\s*injection|sql\s*error)\b",
        "XSS": r"\b(cross-site\s*scripting|xss)\b",
        "RCE": r"\b(remote\s*code\s*execution|command\s*injection)\b",
        "Buffer Overflow": r"\b(buffer\s*overflow)\b",
        "Privilege Escalation": r"\b(privilege\s*escalation)\b",
        "Denial of Service": r"\b(denial\s*of\s*service|dos)\b",
        "Path Traversal": r"\b(path\s*traversal)\b",
        "Command Injection": r"\b(command\s*injection)\b"
    }
    detected_vulnerabilities = [tag for tag, pattern in patterns.items() if re.search(pattern, text, re.IGNORECASE)]
    return detected_vulnerabilities if detected_vulnerabilities else ["General Non-CVE Threat"]

def process_entry(entry):
    """Processes a single RSS feed entry."""
    title = entry.title
    link = entry.link
    published = entry.published
    description = entry.description

    print(f"📌 Processing: {title}")
    cve_ids = extract_cves(description + " " + link +" "+ title )
    cve_str = ", ".join(set(cve_ids)) if cve_ids else "None"
    non_cve_tag = "Yes" if not cve_ids else "No"
    category = ", ".join(classify_non_cve_threats(description)) if not cve_ids else "N/A"

    print(f"🔹 Extracted CVEs: {cve_str}")
    processed_description = preprocess_text(description)
    risk_label, risk_score = classify_threat(processed_description)

    # Insert into Database (Avoid Duplicates)
    cursor.execute("""
        INSERT INTO threats (title, link, published, description, cve_id, risk_label, risk_score, non_cve_tag, category)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (title, link, published, processed_description, cve_str, risk_label, risk_score, non_cve_tag, category))
    conn.commit()

    print("✅ Inserted into database!")

def fetch_feeds():
    """Fetches and processes threat intelligence feeds."""
    for feed_url in FEEDS:
        try:
            print(f"🔍 Fetching from: {feed_url}")
            feed = feedparser.parse(feed_url)
            if not feed.entries:
                print("⚠️ No entries found in feed.")
                continue
            for entry in feed.entries:
                process_entry(entry)
        except Exception as e:
            print(f"⚠️ Error fetching feed {feed_url}: {e}")


# Execute Feed Processing
fetch_feeds()

# Display Stored Data
cursor.execute("SELECT * FROM threats ORDER BY published DESC")
rows = cursor.fetchall()
print("\n📌 Stored Threat Intelligence Data:")
for row in rows:
    print(row)

# Close Database Connection
conn.close()
print("✅ Database connection closed.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


✅ Database setup completed.
🔍 Fetching from: https://feeds.feedburner.com/TheHackersNews
📌 Processing: Hackers Exploit Severe PHP Flaw to Deploy Quasar RAT and XMRig Miners
🔹 Extracted CVEs: CVE-2024-4577
✅ Inserted into database!
📌 Processing: Leaked Black Basta Chats Suggest Russian Officials Aided Leader's Escape from Armenia
🔹 Extracted CVEs: None
✅ Inserted into database!
📌 Processing: Watch This Webinar to Learn How to Eliminate Identity-Based Attacks—Before They Happen
🔹 Extracted CVEs: None
✅ Inserted into database!
📌 Processing: ClearFake Infects 9,300 Sites, Uses Fake reCAPTCHA and Turnstile to Spread Info-Stealers
🔹 Extracted CVEs: None
✅ Inserted into database!
📌 Processing: 5 Identity Threat Detection & Response Must-Haves for Super SaaS Security
🔹 Extracted CVEs: None
✅ Inserted into database!
📌 Processing: Critical mySCADA myPRO Flaws Could Let Attackers Take Over Industrial Control Systems
🔹 Extracted CVEs: None
✅ Inserted into database!
📌 Processing: CISA Warns of Acti

This code is for extracting of CVEids from Blackhawk network Vulcan Cyber Database


In [3]:
import requests
import json
import sqlite3
import time  # To add delays for API rate limiting

class VulcanCVEClient:
    def __init__(self, domain: str, bearer_token: str):
        self.domain = domain
        self.headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "authorization": f"Bearer {bearer_token}"
        }
        self.base_url = f"https://{domain}/api/v2"

    def get_all_cves(self, page_size):
        """Fetches all CVE IDs from the Vulcan API using cursor-based pagination."""
        cve_ids = set()  # Use a set to avoid duplicates
        url = f"{self.base_url}/vulnerabilities"
        cursor = None  # Start with no cursor (first request)

        try:
            while True:
                payload = {
                    "page_size": page_size,  # Dynamic page size
                    "columns": ["cves", "title", "risk_level"],
                }
                if cursor:
                    payload["cursor"] = cursor  # Add cursor for pagination

                print(f"\n🔹 [DEBUG] Fetching page with cursor: {cursor} and page_size: {page_size}")
                response = requests.post(url, headers=self.headers, json=payload)
                print(f"🔹 [DEBUG] Response status: {response.status_code}")

                if response.status_code == 200:
                    data = response.json()
                    vulnerabilities = data.get('data', [])
                    cursor = data.get("next_cursor")  # Ensure using correct key

                    if vulnerabilities:
                        for vuln in vulnerabilities:
                            if 'cves' in vuln and isinstance(vuln['cves'], list):
                                cve_ids.update(vuln['cves'])
                        time.sleep(1)  # Prevent rate limiting
                    else:
                        print("⚠️ [DEBUG] No vulnerabilities found in response.")

                    if not cursor:  # No more pages left
                        print("✅ [DEBUG] No more pages to fetch.")
                        break
                else:
                    print(f"❌ [ERROR] API Request Failed: {response.status_code}, Response: {response.text}")
                    break  # Stop if an error occurs

        except requests.exceptions.RequestException as e:
            print(f"❌ [ERROR] Request Failed: {str(e)}")

        print(f"✅ [DEBUG] Total CVEs fetched: {len(cve_ids)}")
        return list(cve_ids)

# Database setup
DB_NAME = "cve_database.db"
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute("""
    CREATE TABLE IF NOT EXISTS cve_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        cve_id TEXT UNIQUE
    )
""")
conn.commit()

# Function to store CVEs in SQLite database without duplicates
def store_cves_in_db(cve_list):
    if not cve_list:
        print("❌ [ERROR] No CVE IDs found to store!")
        return

    stored_cves = set()
    cursor.execute("SELECT cve_id FROM cve_data")
    for row in cursor.fetchall():
        stored_cves.add(row[0])

    new_cves = [cve for cve in cve_list if cve not in stored_cves]
    if not new_cves:
        print("⚠️ [DEBUG] No new CVEs to insert. All already exist.")
        return

    for cve in new_cves:
        try:
            cursor.execute("INSERT INTO cve_data (cve_id) VALUES (?)", (cve,))
        except sqlite3.IntegrityError:
            pass  # Already handled by in-memory set check

    conn.commit()
    print(f"✅ [DEBUG] {len(new_cves)} new CVEs stored successfully!")

# Function to get count of stored CVEs
def get_cve_count():
    cursor.execute("SELECT COUNT(cve_id) FROM cve_data")
    count = cursor.fetchone()[0]
    print(f"📊 Total CVEs stored in database: {count}")
    return count

# User Inputs
domain = "blackhawknetwork.vulcancyber.com"  # Replace with actual Vulcan domain
token = "DQr1QTzhraiIGEIHCqxp2XTy5fNfd8"  # Replace with actual API token

# Keep fetching until no new CVEs are found, adjusting page size dynamically
client = VulcanCVEClient(domain, token)
page_size = 1000  # Start with a small page size

while page_size<9000:
    cves = client.get_all_cves(page_size)
    before_count = get_cve_count()
    store_cves_in_db(cves)
    after_count = get_cve_count()

    if before_count == after_count:
        print("✅ [DEBUG] No new CVEs found. Stopping fetch loop.")

    page_size = min(page_size + 100, 10000)  # Increase page size up to 10,000



🔹 [DEBUG] Fetching page with cursor: None and page_size: 1000
🔹 [DEBUG] Response status: 200
✅ [DEBUG] No more pages to fetch.
✅ [DEBUG] Total CVEs fetched: 2338
📊 Total CVEs stored in database: 0
✅ [DEBUG] 2338 new CVEs stored successfully!
📊 Total CVEs stored in database: 2338

🔹 [DEBUG] Fetching page with cursor: None and page_size: 1100
🔹 [DEBUG] Response status: 200
✅ [DEBUG] No more pages to fetch.
✅ [DEBUG] Total CVEs fetched: 3039
📊 Total CVEs stored in database: 2338
✅ [DEBUG] 1670 new CVEs stored successfully!
📊 Total CVEs stored in database: 4008

🔹 [DEBUG] Fetching page with cursor: None and page_size: 1200
🔹 [DEBUG] Response status: 200
✅ [DEBUG] No more pages to fetch.
✅ [DEBUG] Total CVEs fetched: 3197
📊 Total CVEs stored in database: 4008
✅ [DEBUG] 2620 new CVEs stored successfully!
📊 Total CVEs stored in database: 6628

🔹 [DEBUG] Fetching page with cursor: None and page_size: 1300
🔹 [DEBUG] Response status: 200
✅ [DEBUG] No more pages to fetch.
✅ [DEBUG] Total CVEs fe

This code is for correlational Analysis.

In [4]:
import sqlite3
import pandas as pd

# Database Paths
threat_db_path = "threat_intel.db"
cve_db_path = "cve_database.db"
remediation_db_path = "remediationdb.db"

# Connect to Databases
conn_threat = sqlite3.connect(threat_db_path)
conn_cve = sqlite3.connect(cve_db_path)
conn_remediation = sqlite3.connect(remediation_db_path)

# Load Data from Threats and CVE Tables (Ensure we select link & summary)
threat_query = "SELECT cve_id, title, link, published, description, risk_label, risk_score, non_cve_tag, category FROM threats"
cve_query = "SELECT cve_id FROM cve_data"

threat_df = pd.read_sql(threat_query, conn_threat)
cve_df = pd.read_sql(cve_query, conn_cve)

# Perform INNER JOIN to keep only threats that have a CVE match
correlated_df = pd.merge(threat_df, cve_df, on="cve_id", how="inner")

# Create New Table in Remediation Database (Include link & summary)
with conn_remediation:
    cursor = conn_remediation.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS correlated_threats (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            link TEXT,
            summary TEXT,
            published TEXT,
            description TEXT,
            cve_id TEXT,
            risk_label TEXT,
            risk_score REAL,
            non_cve_tag TEXT,
            category TEXT
        )
    """)

    # Insert Correlated Data into New Table
    correlated_df.to_sql("correlated_threats", conn_remediation, if_exists="replace", index=False)

# Close Connections
conn_threat.close()
conn_cve.close()
conn_remediation.close()

print("✅ Correlation completed! Data stored in remediationdb.db")

# Display Correlated Results
correlated_df


✅ Correlation completed! Data stored in remediationdb.db


,cve_id,title,link,published,description,risk_label,risk_score,non_cve_tag,category
0,CVE-2024-4577,Hackers Exploit Severe PHP Flaw to Deploy Quas...,https://thehackernews.com/2025/03/hackers-expl...,"Wed, 19 Mar 2025 21:22:00 +0530",Threat actors are exploiting a severe security...,LABEL_1,0.5137,No,N/A
1,CVE-2025-24813,Apache Tomcat Vulnerability Actively Exploited...,https://thehackernews.com/2025/03/apache-tomca...,"Mon, 17 Mar 2025 22:38:00 +0530",A recently disclosed security flaw impacting A...,LABEL_1,0.5100,No,N/A
2,CVE-2025-27363,Meta Warns of FreeType Vulnerability (CVE-2025...,https://thehackernews.com/2025/03/meta-warns-o...,"Thu, 13 Mar 2025 12:43:00 +0530",Meta has warned that a security vulnerability ...,LABEL_0,0.5024,No,N/A
3,CVE-2025-24201,Apple Releases Patch for WebKit Zero-Day Vulne...,https://thehackernews.com/2025/03/apple-releas...,"Wed, 12 Mar 2025 09:32:00 +0530",Apple on Tuesday released a security update to...,LABEL_1,0.5059,No,N/A


In [5]:
import sqlite3
import pandas as pd

# Database Paths
threat_db_path = "threat_intel.db"
cve_db_path = "cve_database.db"
remediation_db_path = "remediationdb.db"

# Connect to Databases
conn_threat = sqlite3.connect(threat_db_path)
conn_cve = sqlite3.connect(cve_db_path)
conn_remediation = sqlite3.connect(remediation_db_path)

# Load Data from Threats and CVE Tables
threat_query = "SELECT * FROM threats"
cve_query = "SELECT cve_id FROM cve_data"

threat_df = pd.read_sql(threat_query, conn_threat)
cve_df = pd.read_sql(cve_query, conn_cve)

# Perform LEFT JOIN and keep only threats where cve_id is matched in cve_data
correlated_df = pd.merge(threat_df, cve_df, on="cve_id", how="inner")

# Create New Table in Remediation Database
with conn_remediation:
    cursor = conn_remediation.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS correlated_threats (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            link TEXT,
            published TEXT,
            description TEXT,
            cve_id TEXT,
            risk_label TEXT,
            risk_score REAL,
            non_cve_tag TEXT,
            category TEXT
        )
    """)

    # Insert Correlated Data into New Table
    correlated_df.to_sql("correlated_threats", conn_remediation, if_exists="replace", index=False)

# Close Connections
conn_threat.close()
conn_cve.close()
conn_remediation.close()

print("Correlation completed! Data stored in remediationdb.db")

# Display Correlated Results
correlated_df


Correlation completed! Data stored in remediationdb.db


,id,title,link,published,description,cve_id,risk_label,risk_score,non_cve_tag,category
0,1,Hackers Exploit Severe PHP Flaw to Deploy Quas...,https://thehackernews.com/2025/03/hackers-expl...,"Wed, 19 Mar 2025 21:22:00 +0530",Threat actors are exploiting a severe security...,CVE-2024-4577,LABEL_1,0.5137,No,N/A
1,17,Apache Tomcat Vulnerability Actively Exploited...,https://thehackernews.com/2025/03/apache-tomca...,"Mon, 17 Mar 2025 22:38:00 +0530",A recently disclosed security flaw impacting A...,CVE-2025-24813,LABEL_1,0.5100,No,N/A
2,34,Meta Warns of FreeType Vulnerability (CVE-2025...,https://thehackernews.com/2025/03/meta-warns-o...,"Thu, 13 Mar 2025 12:43:00 +0530",Meta has warned that a security vulnerability ...,CVE-2025-27363,LABEL_0,0.5024,No,N/A
3,40,Apple Releases Patch for WebKit Zero-Day Vulne...,https://thehackernews.com/2025/03/apple-releas...,"Wed, 12 Mar 2025 09:32:00 +0530",Apple on Tuesday released a security update to...,CVE-2025-24201,LABEL_1,0.5059,No,N/A


In [6]:
import sqlite3
import pandas as pd

# Path to Remediation Database
remediation_db_path = "remediationdb.db"

# Connect to Remediation Database
conn_remediation = sqlite3.connect(remediation_db_path)

# Query to fetch all records from correlated_threats
query = "SELECT * FROM correlated_threats"

# Load data into a DataFrame
df_remediation = pd.read_sql(query, conn_remediation)

# Close connection
conn_remediation.close()

# Display the data
print(df_remediation)

# Optionally display only specific columns
# print(df_remediation[['cve_id', 'title', 'link', 'summary']])


   id                                              title  \
0   1  Hackers Exploit Severe PHP Flaw to Deploy Quas...   
1  17  Apache Tomcat Vulnerability Actively Exploited...   
2  34  Meta Warns of FreeType Vulnerability (CVE-2025...   
3  40  Apple Releases Patch for WebKit Zero-Day Vulne...   

                                                link  \
0  https://thehackernews.com/2025/03/hackers-expl...   
1  https://thehackernews.com/2025/03/apache-tomca...   
2  https://thehackernews.com/2025/03/meta-warns-o...   
3  https://thehackernews.com/2025/03/apple-releas...   

                         published  \
0  Wed, 19 Mar 2025 21:22:00 +0530   
1  Mon, 17 Mar 2025 22:38:00 +0530   
2  Thu, 13 Mar 2025 12:43:00 +0530   
3  Wed, 12 Mar 2025 09:32:00 +0530   

                                         description          cve_id  \
0  Threat actors are exploiting a severe security...   CVE-2024-4577   
1  A recently disclosed security flaw impacting A...  CVE-2025-24813   
2  Meta has

This code is for creation of post gres and migration of data from RemediateDB to Postgres.

In [7]:
!apt-get -y install postgresql postgresql-client
!pip install psycopg2


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql postgresql-14 postgresql-client postgresql-client-14 postgresql-client-common
  postgresql-common ssl-cert sysstat
0 upgraded, 14 newly installed, 0 to remove and 29 not upgraded.
Need to get 18.4 MB of archives.
After this operation, 52.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 logrotate amd64 3.19.0-1ubuntu1.1 [54.3 kB]
Get:2 http://archive.ubuntu.com/ubun

In [8]:
!service postgresql start
!sudo -u postgres psql -tc "SELECT 1 FROM pg_database WHERE datname = 'cve_db'" | grep -q 1 || \
    sudo -u postgres psql -c "CREATE DATABASE cve_db;"
!sudo -u postgres psql -tc "SELECT 1 FROM pg_roles WHERE rolname = 'vulcan'" | grep -q 1 || \
    sudo -u postgres psql -c "CREATE USER vulcan WITH PASSWORD 'securepassword';"
!sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE cve_db TO vulcan;"

 * Starting PostgreSQL 14 database server
   ...done.
CREATE DATABASE
CREATE ROLE
GRANT


This is creation **(Modification is Require) where it should check for Schema**

In [9]:
import psycopg2

DB_CONFIG = {
    "dbname": "cve_db",
    "user": "vulcan",
    "password": "securepassword",
    "host": "localhost",
    "port": 5432
}

def create_table():
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS cve_tracking (
            id SERIAL PRIMARY KEY,
            cve_id TEXT UNIQUE NOT NULL,
            status TEXT DEFAULT 'new',
            added_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
    """)
    conn.commit()
    conn.close()
    print("✅ PostgreSQL table created.")

create_table()


✅ PostgreSQL table created.


Migration Code from Sqlite to Postgres


In [10]:
import sqlite3
import psycopg2

# Connect to SQLite (Remediation Database)
sqlite_conn = sqlite3.connect("remediationdb.db")
sqlite_cursor = sqlite_conn.cursor()

# Connect to PostgreSQL
DB_CONFIG = {
    "dbname": "cve_db",
    "user": "vulcan",
    "password": "securepassword",
    "host": "localhost",
    "port": 5432
}
pg_conn = psycopg2.connect(**DB_CONFIG)
pg_cursor = pg_conn.cursor()

# Ensure PostgreSQL table has title and description columns
pg_cursor.execute("""
    ALTER TABLE cve_tracking
    ADD COLUMN IF NOT EXISTS title TEXT,
    ADD COLUMN IF NOT EXISTS description TEXT;
""")
pg_conn.commit()
print("✅ Verified PostgreSQL table structure.")

# Fetch unique CVEs along with title and description from SQLite
sqlite_cursor.execute("SELECT DISTINCT cve_id, title, description FROM correlated_threats")
cve_data = sqlite_cursor.fetchall()

print(f"✅ Found {len(cve_data)} CVEs in SQLite remediation database.")

# Insert only new CVEs into PostgreSQL or update existing ones
for cve_id, title, description in cve_data:
    pg_cursor.execute("""
        INSERT INTO cve_tracking (cve_id, title, description, status)
        VALUES (%s, %s, %s, 'new')
        ON CONFLICT (cve_id) DO UPDATE
        SET title = EXCLUDED.title,
            description = EXCLUDED.description;
    """, (cve_id, title, description))

pg_conn.commit()
print("✅ Migration completed! CVEs, titles, and descriptions added to PostgreSQL.")

# Close database connections
sqlite_conn.close()
pg_conn.close()
print("✅ Database connections closed.")


✅ Verified PostgreSQL table structure.
✅ Found 4 CVEs in SQLite remediation database.
✅ Migration completed! CVEs, titles, and descriptions added to PostgreSQL.
✅ Database connections closed.


In [11]:
def fetch_unremediated_cves():
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()
    cursor.execute("SELECT cve_id FROM cve_tracking WHERE status = 'new'")
    cve_ids = [row[0] for row in cursor.fetchall()]
    conn.close()
    return cve_ids

print("🔍 Unremediated CVEs:", fetch_unremediated_cves())


🔍 Unremediated CVEs: ['CVE-2024-4577', 'CVE-2025-24813', 'CVE-2025-27363', 'CVE-2025-24201']


Testing Purpose Code

In [12]:
import psycopg2

# Your PostgreSQL connection details
DB_CONFIG = {
    "dbname": "cve_db",
    "user": "vulcan",
    "password": "securepassword",
    "host": "localhost",
    "port": 5432
}

def print_table_contents():
    try:
        print("🔄 Connecting to database...")
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        print("✅ Connected to the database.")

        # Select all rows from the cve_tracking table
        cursor.execute("SELECT * FROM cve_tracking;")
        rows = cursor.fetchall()

        # Print column names
        colnames = [desc[0] for desc in cursor.description]
        print("📌 Table Columns:", colnames)

        # Print table data
        for row in rows:
            print(row)

    except Exception as e:
        print("❌ Database error:", e)
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

# Run the function
print_table_contents()


🔄 Connecting to database...
✅ Connected to the database.
📌 Table Columns: ['id', 'cve_id', 'status', 'added_at', 'title', 'description']
(1, 'CVE-2024-4577', 'new', datetime.datetime(2025, 3, 19, 20, 14, 47, 405249), 'Hackers Exploit Severe PHP Flaw to Deploy Quasar RAT and XMRig Miners', 'Threat actors are exploiting a severe security flaw in PHP to deliver cryptocurrency miners and remote access trojans (RATs) like Quasar RAT.\nThe vulnerability, assigned the CVE identifier CVE-2024-4577, refers to an argument injection vulnerability in PHP affecting Windows-based systems running in CGI mode that could allow remote attackers to run arbitrary code.\nCybersecurity company')
(2, 'CVE-2025-24813', 'new', datetime.datetime(2025, 3, 19, 20, 14, 47, 405249), 'Apache Tomcat Vulnerability Actively Exploited Just 30 Hours After Public Disclosure', 'A recently disclosed security flaw impacting Apache Tomcat has come under active exploitation in the wild following the release&nbsp;of a public pr

Epss and CVSS Score Linking from National Vulnerability Database

In [13]:
import psycopg2
import requests
from datetime import datetime

# PostgreSQL Connection Details
DB_CONFIG = {
   "dbname": "cve_db",
    "user": "vulcan",
    "password": "securepassword",
    "host": "localhost",
    "port": 5432
}

# Ensure required columns exist in the table
def ensure_columns_exist():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # Check existing columns in the table
        cursor.execute("""
            SELECT column_name FROM information_schema.columns
            WHERE table_name = 'cve_tracking';
        """)
        existing_columns = {row[0] for row in cursor.fetchall()}

        # Columns to add if missing
        columns_to_add = []
        if "cvss_score" not in existing_columns:
            columns_to_add.append("ADD COLUMN cvss_score FLOAT")
        if "epss_score" not in existing_columns:
            columns_to_add.append("ADD COLUMN epss_score FLOAT")
        if "epss_percentile" not in existing_columns:
            columns_to_add.append("ADD COLUMN epss_percentile FLOAT")
        if "updated_at" not in existing_columns:
            columns_to_add.append("ADD COLUMN updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP")

        # Execute ALTER TABLE if new columns are needed
        if columns_to_add:
            alter_query = f"ALTER TABLE cve_tracking {', '.join(columns_to_add)};"
            cursor.execute(alter_query)
            conn.commit()
            print("✅ Missing columns added successfully!")

    except Exception as e:
        print("Error checking/updating table structure:", e)
    finally:
        if conn:
            cursor.close()
            conn.close()

# Fetch CVSS Score from NVD API
def get_cvss_score(cve_id):
    url = f"https://services.nvd.nist.gov/rest/json/cves/2.0?cveId={cve_id}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        try:
            cvss_v3 = data['vulnerabilities'][0]['cve']['metrics']['cvssMetricV31'][0]['cvssData']['baseScore']
            return cvss_v3
        except (KeyError, IndexError):
            return None
    return None

# Fetch EPSS Score from FIRST API
def get_epss_score(cve_id):
    url = f"https://api.first.org/data/v1/epss?cve={cve_id}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        try:
            epss_entry = data['data'][0] if isinstance(data['data'], list) else data['data'].get(cve_id, {})
            epss_score = epss_entry.get('epss', None)
            epss_percentile = epss_entry.get('percentile', None)
            return epss_score, epss_percentile
        except (KeyError, IndexError):
            return None, None
    return None, None

# Connect to PostgreSQL and Update CVE Data
def update_cve_scores():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # Ensure necessary columns exist
        ensure_columns_exist()

        # Step 1: Fetch CVE IDs from Database
        cursor.execute("SELECT cve_id FROM cve_tracking WHERE status = 'new'")
        cve_list = cursor.fetchall()

        if not cve_list:
            print("No new CVE IDs to process.")
            return

        for cve in cve_list:
            cve_id = cve[0]
            print(f"Processing {cve_id}...")

            # Step 2: Fetch CVSS and EPSS Scores
            cvss_score = get_cvss_score(cve_id)
            epss_score, epss_percentile = get_epss_score(cve_id)

            # Step 3: Update Database
            cursor.execute("""
                UPDATE cve_tracking
                SET cvss_score = %s, epss_score = %s, epss_percentile = %s, status = 'updated', updated_at = %s
                WHERE cve_id = %s
            """, (cvss_score, epss_score, epss_percentile, datetime.now(), cve_id))

            print(f"✅ Updated {cve_id}: CVSS={cvss_score}, EPSS={epss_score}, Percentile={epss_percentile}")

        conn.commit()
        print("✅ Database updated successfully!")

    except Exception as e:
        print("Database error:", e)
    finally:
        if conn:
            cursor.close()
            conn.close()

# Run the script
update_cve_scores()


✅ Missing columns added successfully!
Processing CVE-2024-4577...
✅ Updated CVE-2024-4577: CVSS=9.8, EPSS=0.943760000, Percentile=0.999610000
Processing CVE-2025-24813...
✅ Updated CVE-2025-24813: CVSS=9.8, EPSS=0.801570000, Percentile=0.990550000
Processing CVE-2025-27363...
✅ Updated CVE-2025-27363: CVSS=8.1, EPSS=0.002600000, Percentile=0.463560000
Processing CVE-2025-24201...
✅ Updated CVE-2025-24201: CVSS=8.8, EPSS=0.001580000, Percentile=0.333600000
✅ Database updated successfully!


In [ ]:
Testing Purpose

In [14]:
import psycopg2

# Your PostgreSQL connection details
DB_CONFIG = {
    "dbname": "cve_db",
    "user": "vulcan",
    "password": "securepassword",
    "host": "localhost",
    "port": 5432
}

def print_table_contents():
    try:
        print("🔄 Connecting to database...")
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        print("✅ Connected to the database.")

        # Select all rows from the cve_tracking table
        cursor.execute("SELECT * FROM cve_tracking;")
        rows = cursor.fetchall()

        # Print column names
        colnames = [desc[0] for desc in cursor.description]
        print("📌 Table Columns:", colnames)

        # Print table data
        for row in rows:
            print(row)

    except Exception as e:
        print("❌ Database error:", e)
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

# Run the function
print_table_contents()


🔄 Connecting to database...
✅ Connected to the database.
📌 Table Columns: ['id', 'cve_id', 'status', 'added_at', 'title', 'description', 'cvss_score', 'epss_score', 'epss_percentile', 'updated_at']
(1, 'CVE-2024-4577', 'updated', datetime.datetime(2025, 3, 19, 20, 14, 47, 405249), 'Hackers Exploit Severe PHP Flaw to Deploy Quasar RAT and XMRig Miners', 'Threat actors are exploiting a severe security flaw in PHP to deliver cryptocurrency miners and remote access trojans (RATs) like Quasar RAT.\nThe vulnerability, assigned the CVE identifier CVE-2024-4577, refers to an argument injection vulnerability in PHP affecting Windows-based systems running in CGI mode that could allow remote attackers to run arbitrary code.\nCybersecurity company', 9.8, 0.94376, 0.99961, datetime.datetime(2025, 3, 19, 20, 14, 59, 650049))
(2, 'CVE-2025-24813', 'updated', datetime.datetime(2025, 3, 19, 20, 14, 47, 405249), 'Apache Tomcat Vulnerability Actively Exploited Just 30 Hours After Public Disclosure', 'A r

VirusTotal Threat Intelligence Platform Integration

In [ ]:
import psycopg2
import requests
import json

# PostgreSQL Database Configuration
DB_CONFIG = {
    "dbname": "cve_db",
    "user": "vulcan",
    "password": "securepassword",
    "host": "localhost",
    "port": 5432
}

# VirusTotal API Key (Replace with your actual key)
VT_API_KEY = "cf08ae6fb271e4888555779f6fc072f4324c123ff8ad59581cf5021a36fdd19c"
VT_API_URL = "https://www.virustotal.com/api/v3/search?query="

def get_virustotal_report(cve_id):
    """Fetch VirusTotal report for a given CVE ID."""
    headers = {
        "x-apikey": VT_API_KEY
    }
    response = requests.get(VT_API_URL + cve_id, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return json.dumps(data)  # Convert response to JSON string
    else:
        return None

def update_database_with_vt():
    """Fetch VirusTotal reports and update the database."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # Fetch all CVE IDs that need VirusTotal reports
        cursor.execute("SELECT cve_id FROM cve_tracking WHERE virustotal_report IS NULL")
        cve_ids = cursor.fetchall()

        for cve in cve_ids:
            cve_id = cve[0]
            print(f"Fetching VirusTotal report for: {cve_id}")

            report = get_virustotal_report(cve_id)

            if report:
                cursor.execute("UPDATE cve_tracking SET virustotal_report = %s WHERE cve_id = %s", (report, cve_id))
                print(f"Updated {cve_id} with VirusTotal data.")

        conn.commit()
        print("✅ Database updated successfully.")

    except Exception as e:
        print(f"❌ Database error: {e}")

    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

# Run the update function
update_database_with_vt()


❌ Database error: column "virustotal_report" does not exist
LINE 1: SELECT cve_id FROM cve_tracking WHERE virustotal_report IS N...
                                              ^



Testing Purpose Display!

In [15]:
import psycopg2

# Your PostgreSQL connection details
DB_CONFIG = {
    "dbname": "cve_db",
    "user": "vulcan",
    "password": "securepassword",
    "host": "localhost",
    "port": 5432
}

def print_table_contents():
    try:
        print("🔄 Connecting to database...")
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        print("✅ Connected to the database.")

        # Select all rows from the cve_tracking table
        cursor.execute("SELECT * FROM cve_tracking;")
        rows = cursor.fetchall()

        # Print column names
        colnames = [desc[0] for desc in cursor.description]
        print("📌 Table Columns:", colnames)

        # Print table data
        for row in rows:
            print(row)

    except Exception as e:
        print("❌ Database error:", e)
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

# Run the function
print_table_contents()


🔄 Connecting to database...
✅ Connected to the database.
📌 Table Columns: ['id', 'cve_id', 'status', 'added_at', 'title', 'description', 'cvss_score', 'epss_score', 'epss_percentile', 'updated_at']
(1, 'CVE-2024-4577', 'updated', datetime.datetime(2025, 3, 19, 20, 14, 47, 405249), 'Hackers Exploit Severe PHP Flaw to Deploy Quasar RAT and XMRig Miners', 'Threat actors are exploiting a severe security flaw in PHP to deliver cryptocurrency miners and remote access trojans (RATs) like Quasar RAT.\nThe vulnerability, assigned the CVE identifier CVE-2024-4577, refers to an argument injection vulnerability in PHP affecting Windows-based systems running in CGI mode that could allow remote attackers to run arbitrary code.\nCybersecurity company', 9.8, 0.94376, 0.99961, datetime.datetime(2025, 3, 19, 20, 14, 59, 650049))
(2, 'CVE-2025-24813', 'updated', datetime.datetime(2025, 3, 19, 20, 14, 47, 405249), 'Apache Tomcat Vulnerability Actively Exploited Just 30 Hours After Public Disclosure', 'A r

This part of Code is gets AI patch Suggestions from Microsoft Co-piot!

In [16]:
import psycopg2
from openai import AzureOpenAI

# Database Configuration
DB_CONFIG = {
    "dbname": "cve_db",
    "user": "vulcan",
    "password": "securepassword",
    "host": "localhost",
    "port": 5432
}

# Azure OpenAI Configuration
AZURE_OPENAI_CONFIG = {
    "api_key": "eae06bab80034c3c86f0fa1ab526b2e3",  # Replace with real API key
    "azure_endpoint": "https://bhn-dev-azureopenai.openai.azure.com/",
    "api_version": "2023-12-01-preview"
}

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key=AZURE_OPENAI_CONFIG["api_key"],
    azure_endpoint=AZURE_OPENAI_CONFIG["azure_endpoint"],
    api_version=AZURE_OPENAI_CONFIG["api_version"]
)

def ensure_columns_exist():
    """Ensure the ai_patch_suggestion column exists in the cve_tracking table."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        cursor.execute("""
            SELECT column_name FROM information_schema.columns
            WHERE table_name = 'cve_tracking';
        """)
        existing_columns = {row[0] for row in cursor.fetchall()}

        if "ai_patch_suggestion" not in existing_columns:
            cursor.execute("ALTER TABLE cve_tracking ADD COLUMN ai_patch_suggestion TEXT;")
            conn.commit()
            print("✅ Column 'ai_patch_suggestion' added successfully!")

    except Exception as e:
        print("❌ Error ensuring table structure:", e)
    finally:
        if conn:
            cursor.close()
            conn.close()

def generate_cve_description(cve_id):
    """Generate an AI-based CVE description."""
    try:
        print(f"🟡 Generating description for {cve_id}...")

        completion = client.chat.completions.create(
            model="gpt-35-turbo-2",
            messages=[
                {"role": "system", "content": "You are an AI security expert providing detailed CVE descriptions."},
                {"role": "user", "content": f"Describe the security vulnerability for {cve_id}."}
            ],
        )

        response = completion.choices[0].message.content  # ✅ Fixed dot notation
        print(f"✅ CVE Description: {response}")  # Debugging
        return response.strip()

    except Exception as e:
        print(f"❌ Error generating CVE description for {cve_id}: {e}")
        return "No detailed description available."

def generate_patch_suggestion(cve_id):
    """Generate an AI patch suggestion for a given CVE ID."""
    try:
        print(f"🟡 Generating AI patch for CVE: {cve_id}")  # Debugging

        # Generate a description first
        cve_description = generate_cve_description(cve_id)
        print(f"🔹 CVE Description: {cve_description}")  # Debugging

        # Call OpenAI API for patch suggestion
        completion = client.chat.completions.create(
            model="gpt-35-turbo-2",
            messages=[
                {"role": "system", "content": "You are an AI security expert providing patch suggestions."},
                {"role": "user", "content": f"Provide a patch suggestion for this vulnerability: {cve_description}"}
            ],
        )

        response = completion.choices[0].message.content  # ✅ Fixed dot notation
        print(f"✅ AI Patch Suggestion: {response}")  # Debugging

        return response.strip() if response else None

    except Exception as e:
        print(f"❌ Error generating AI patch suggestion for {cve_id}: {e}")
        return None

def fetch_all_cves():
    """Retrieve all CVEs that do not have AI patch suggestions."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        cursor.execute("""
            SELECT cve_id FROM cve_tracking
            WHERE ai_patch_suggestion IS NULL OR ai_patch_suggestion = '';
        """)
        cves = cursor.fetchall()

        print(f"🟢 Found {len(cves)} CVEs needing patch suggestions: {cves}")  # Debugging
        return [cve[0] for cve in cves]

    except Exception as e:
        print("❌ Error fetching CVEs:", e)
        return []
    finally:
        if conn:
            cursor.close()
            conn.close()

def update_ai_patch_suggestion(cve_id, patch_suggestion):
    """Update the database with the AI-generated patch suggestion."""
    try:
        if not patch_suggestion:
            print(f"⚠️ No AI patch suggestion generated for {cve_id}. Skipping update.")  # Debugging
            return

        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        print(f"🟡 Updating database for CVE {cve_id} with patch: {patch_suggestion[:100]}...")  # Debugging

        cursor.execute("""
            UPDATE cve_tracking
            SET ai_patch_suggestion = %s, updated_at = CURRENT_TIMESTAMP
            WHERE cve_id = %s;
        """, (patch_suggestion, cve_id))

        conn.commit()
        print(f"✅ AI Patch Suggestion updated for CVE ID {cve_id}")

    except Exception as e:
        print(f"❌ Error updating AI patch suggestion for CVE ID {cve_id}: {e}")

    finally:
        if conn:
            cursor.close()
            conn.close()

def process_all_cves():
    """Process all CVEs and update them with AI patch suggestions."""
    ensure_columns_exist()  # Ensure the column exists
    cves = fetch_all_cves()

    if not cves:
        print("✅ No CVEs found that need AI patch suggestions.")
        return

    print(f"📌 Processing {len(cves)} CVEs...")

    for cve_id in cves:
        print(f"🔍 Generating patch for {cve_id}...")
        patch_suggestion = generate_patch_suggestion(cve_id)
        if patch_suggestion:
            update_ai_patch_suggestion(cve_id, patch_suggestion)

    print("🎉 All AI patch suggestions updated successfully!")

if __name__ == "__main__":
    process_all_cves()


✅ Column 'ai_patch_suggestion' added successfully!
🟢 Found 4 CVEs needing patch suggestions: [('CVE-2024-4577',), ('CVE-2025-24813',), ('CVE-2025-27363',), ('CVE-2025-24201',)]
📌 Processing 4 CVEs...
🔍 Generating patch for CVE-2024-4577...
🟡 Generating AI patch for CVE: CVE-2024-4577
🟡 Generating description for CVE-2024-4577...
✅ CVE Description: CVE-2024-4577 is a security vulnerability that exists in a popular web application framework. The vulnerability allows an attacker to perform a Cross-Site Scripting (XSS) attack on the application. This occurs due to insufficient input validation of user-controlled data that is not properly sanitized before being rendered back to the user. An attacker could exploit this flaw by injecting malicious scripts into the application, which could potentially lead to the theft of sensitive information, session hijacking, or other malicious activities. This vulnerability poses a significant risk to the confidentiality and integrity of the application a

This part of code helps in visual Display of the correlated and AI Patch Suggestion

Interactive Dashboard for the above.


In [17]:
!pip install gradio psycopg2 pandas plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.8 MB/s eta 0:00:00


Tryouts

In [22]:
import psycopg2
import pandas as pd
import gradio as gr
import plotly.express as px

# Database Configuration
DB_CONFIG = {
    "dbname": "cve_db",
    "user": "vulcan",
    "password": "securepassword",
    "host": "localhost",  # Change if using an external DB
    "port": 5432
}

def fetch_cve_data():
    """Fetch CVE records and return as a Pandas DataFrame"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        query = """
        SELECT cve_id, title, description, status, cvss_score, epss_score, epss_percentile, updated_at, ai_patch_suggestion
        FROM cve_tracking;
        """
        cursor.execute(query)
        records = cursor.fetchall()

        if not records:
            return pd.DataFrame(columns=["CVE ID", "Title", "Description", "Status", "CVSS Score", "EPSS Score", "EPSS Percentile", "Updated At", "AI Patch Suggestion"])

        df = pd.DataFrame(records, columns=["CVE ID", "Title", "Description", "Status", "CVSS Score", "EPSS Score", "EPSS Percentile", "Updated At", "AI Patch Suggestion"])

        # Apply formatting to fix newline issue
        df["Description"] = df["Description"].apply(lambda x: x.replace("\n", "<br>") if isinstance(x, str) else x)
        df["AI Patch Suggestion"] = df["AI Patch Suggestion"].apply(lambda x: x.replace("\n", "<br>") if isinstance(x, str) else x)

        return df

    except Exception as e:
        return pd.DataFrame({"Error": [str(e)]})

def generate_visuals():
    """Generate visualizations and display CVE data"""
    df = fetch_cve_data()

    if df.empty:
        return None, None, None, "No Data Available", "No AI Patch Available"

    # Bar chart for CVSS Scores
    cvss_chart = px.bar(df, x="CVE ID", y="CVSS Score", color="Status", title="CVSS Score by CVE", height=400)

    # Scatter Plot for EPSS Score vs Percentile
    epss_chart = px.scatter(df, x="EPSS Score", y="EPSS Percentile", color="CVE ID",
                            title="EPSS Score vs EPSS Percentile", height=400, size_max=10)

    # Display latest CVE details
    latest_cve = df.iloc[-1] if not df.empty else None
    description_text = latest_cve["Description"] if latest_cve is not None else "No description available."
    ai_patch_text = latest_cve["AI Patch Suggestion"] if latest_cve is not None else "No AI patch available."

    return df, cvss_chart, epss_chart, description_text, ai_patch_text

# Gradio Interface
with gr.Blocks() as dashboard:
    gr.Markdown("# 🔥 Threat Intel - CVE Dashboard")
    gr.Markdown("A visual analytics dashboard for security vulnerabilities.")

    fetch_button = gr.Button("Fetch CVE Data")

    # Scrollable Data Table
    data_display = gr.HTML(label="CVE Data Table")

    with gr.Row():
        graph_cvss = gr.Plot(label="CVSS Score Distribution")
        graph_epss = gr.Plot(label="EPSS Score Analysis")

    with gr.Row():
        description_box = gr.Textbox(label="CVE Description", interactive=False)
        patch_box = gr.Textbox(label="AI Patch Suggestion", interactive=False)

    def update_dashboard():
        df, cvss_chart, epss_chart, description_text, ai_patch_text = generate_visuals()
        table_html = df.to_html(escape=False, index=False)  # **Fix applied here!**
        return table_html, cvss_chart, epss_chart, description_text, ai_patch_text

    fetch_button.click(fn=update_dashboard, outputs=[data_display, graph_cvss, graph_epss, description_box, patch_box])

dashboard.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fcb6e3cc77b6d19be9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
